# QUAAF Returns

#### Import Packages

In [1]:
import numpy as np                      # Linear algebra
import pandas as pd                     # Dataframes
from collections import defaultdict     # For dictionaries of dataframes

pd.options.mode.chained_assignment = None  # default='warn'

#### Load Holding Data

In [2]:
filename = 'holdings-newcol_DD.xlsx'

holdings = pd.ExcelFile(filename)

sheet_names = holdings.sheet_names  # Get list of sheets
sheet_names                         # Print list of sheets

['summary',
 'growth_cad',
 'growth_usd',
 'market_neutral_cad',
 'market_neutral_usd',
 'sustainability_cad',
 'sustainability_usd',
 'thematic_cad',
 'thematic_usd',
 'special_situations_cad',
 'special_situations_usd',
 'conservative_tactical_cad',
 'conservative_tactical_usd',
 'original']

In [4]:
# Load Summary Data
holdings_summary = pd.read_excel(filename, sheet_name = sheet_names[0], header = 5)

In [5]:
# Inspect Data
holdings_summary

,Month,Year,Date,Check,QUAAF,Total Holdings (CAD),CAD Investments,CAD Equivalent of USD Investments,Exchange Rate,USD Investments,Growth (Total),Growth (Total)[trans],Growth (Total)[Cost Basis],Growth (Total)[Unrealized Gain (Loss)],Growth (Total)[Realized Gain (Loss)],Market Neutral (Total),Market Neutral (Total)[trans],Market Neutral (Total)[Cost Basis],Market Neutral (Total)[Unrealized Gain (Loss)],Market Neutral (Total)[Realized Gain (Loss)],Sustainability (Total),Sustainability (Total)[trans],Sustainability (Total)[Cost Basis],Sustainability (Total)[Unrealized Gain (Loss)],Sustainability (Total)[Realized Gain (Loss)],Thematic (Total),Thematic (Total)[trans],Thematic (Total)[Cost Basis],Thematic (Total)[Unrealized Gain (Loss)],Thematic (Total)[Realized Gain (Loss)],Special Situations (Total),Special Situations (Total)[trans],Special Situations (Total)[Cost Basis],Special Situations (Total)[Unrealized Gain (Loss)],Special Situations (Total)[Realized Gain (Loss)],Conservative Tactical (Total),Conservative Tactical (Total)[trans],Growth (CAD),Growth (CAD)[trans],Growth (CAD)[Cost Basis],...,Market Neutral (USD),Market Neutral (USD)[trans],Market Neutral (USD)[Cost Basis],Market Neutral (USD)[Unrealized Gain (Loss)],Market Neutral (USD)[Realized Gain (Loss)],Sustainability (CAD),Sustainability (CAD)[trans],Sustainability (CAD)[Cost Basis],Sustainability (CAD)[Unrealized Gain (Loss)],Sustainability (CAD)[Realized Gain (Loss)],Sustainability (USD),Sustainability (USD)[trans],Sustainability (USD)[Cost Basis],Sustainability (USD)[Unrealized Gain (Loss)],Sustainability (USD)[Realized Gain (Loss)],Thematic (CAD),Thematic (CAD)[trans],Thematic (CAD)[Cost Basis],Thematic (CAD)[Unrealized Gain (Loss)],Thematic (CAD)[Realized Gain (Loss)],Thematic (USD),Thematic (USD)[trans],Thematic (USD)[Cost Basis],Thematic (USD)[Unrealized Gain (Loss)],Thematic (USD)[Realized Gain (Loss)],Special Situations (CAD),Special Situations (CAD)[trans],Special Situations (CAD)[Cost Basis],Special Situations (CAD)[Unrealized Gain (Loss)],Special Situations (CAD)[Realized Gain (Loss)],Special Situations (USD),Special Situations (USD)[trans],Special Situations (USD)[Cost Basis],Special Situations (USD)[Unrealized Gain (Loss)],Special Situations (USD)[Realized Gain (Loss)],Conservative Tactical (CAD),Conservative Tactical (USD),Conservative Tactical (CAD)[Cost Basis],Conservative Tactical (CAD)[Unrealized Gain (Loss)],Conservative Tactical (CAD)[Realized Gain (Loss)]
0,2012-01-31 00:00:00,2012-01-31,2012-01-31,0,0.0,0.000,0.00,0.000,NaN,0,0,0.00,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0.000,0.0000,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,0.000,NaN,0,0.00,NaN,...,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN
1,2012-02-29 00:00:00,2012-02-29,2012-02-29,0,25000.0,25000.000,25000.00,0.000,NaN,0,0,0.00,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0.000,0.0000,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,25000.000,NaN,0,0.00,NaN,...,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,25000.00,0,NaN,NaN,NaN
2,2012-03-31 00:00:00,2012-03-31,2012-03-31,0,175065.0,175065.000,175065.00,0.000,NaN,0,0,0.00,NaN,NaN,NaN,20027.0,20000.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0.000,0.0000,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,155038.000,NaN,0,0.00,NaN,...,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,155038.00,0,NaN,NaN,NaN
3,2012-04-30 00:00:00,2012-04-30,2012-04-30,0,174766.0,174766.000,174766.00,0.000,NaN,0,0,0.00,NaN,NaN,NaN,59714.0,40000.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0.000,0.0000,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,115052.000,NaN,0,0.00,NaN,...,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.00,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,115052.00,0,NaN,NaN,NaN
4,2012-05-31 00:00:00,2012-05-31,2012-05-31,0,174885.0,174885.000,174885.00,0.000,NaN,0,0

In [6]:
for col in holdings_summary.columns:
    print(col)

Month
Year
Date
Check
QUAAF
Total Holdings (CAD)
CAD Investments
CAD Equivalent of USD Investments
Exchange Rate
USD Investments
Growth (Total)
Growth (Total)[trans]
Growth (Total)[Cost Basis]
Growth (Total)[Unrealized Gain (Loss)]
Growth (Total)[Realized Gain (Loss)]
Market Neutral (Total)
Market Neutral (Total)[trans]
Market Neutral (Total)[Cost Basis]
Market Neutral (Total)[Unrealized Gain (Loss)]
Market Neutral (Total)[Realized Gain (Loss)]
Sustainability (Total)
Sustainability (Total)[trans]
Sustainability (Total)[Cost Basis]
Sustainability (Total)[Unrealized Gain (Loss)]
Sustainability (Total)[Realized Gain (Loss)]
Thematic (Total)
Thematic (Total)[trans]
Thematic (Total)[Cost Basis]
Thematic (Total)[Unrealized Gain (Loss)]
Thematic (Total)[Realized Gain (Loss)]
Special Situations (Total)
Special Situations (Total)[trans]
Special Situations (Total)[Cost Basis]
Special Situations (Total)[Unrealized Gain (Loss)]
Special Situations (Total)[Realized Gain (Loss)]
Conservative Tactical

#### Data Cleaning

In [7]:
# Extract columns relevant for return by fund group
col_keep = ['Date','Growth (Total)', 'Growth (Total)[trans]',
            'Market Neutral (Total)', 'Market Neutral (Total)[trans]',
            'Sustainability (Total)', 'Sustainability (Total)[trans]',
            'Thematic (Total)', 'Thematic (Total)[trans]',
            'Special Situations (Total)', 'Special Situations (Total)[trans]',
            'Conservative Tactical (Total)', 'Conservative Tactical (Total)[trans]']

holdings_summary = holdings_summary[holdings_summary.columns[holdings_summary.columns.isin(col_keep)]]

In [8]:
# Reset index to date
holdings_summary.set_index('Date', inplace=True)
holdings_summary.index.name = None

In [9]:
# Replace NaNs with 0s (No position = $0 position)
holdings_summary = holdings_summary.fillna(0)

In [10]:
# Inspect cleaned dataframe
holdings_summary

,Growth (Total),Growth (Total)[trans],Market Neutral (Total),Market Neutral (Total)[trans],Sustainability (Total),Sustainability (Total)[trans],Thematic (Total),Thematic (Total)[trans],Special Situations (Total),Special Situations (Total)[trans],Conservative Tactical (Total),Conservative Tactical (Total)[trans]
2012-01-31,0,0.00,0.0,0.0,0,0,0.000,0.0000,0,0.0,0.000,0.0
2012-02-29,0,0.00,0.0,0.0,0,0,0.000,0.0000,0,0.0,25000.000,0.0
2012-03-31,0,0.00,20027.0,20000.0,0,0,0.000,0.0000,0,0.0,155038.000,0.0
2012-04-30,0,0.00,59714.0,40000.0,0,0,0.000,0.0000,0,0.0,115052.000,0.0
2012-05-31,0,0.00,59718.0,0.0,0,0,0.000,0.0000,0,0.0,115167.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-30,167594,0.00,303558.0,0.0,0,0,23468.366,0.0000,0,0.0,43315.000,0.0
2020-12-31,175801,0.00,312494.0,0.0,0,0,31144.475,2733.6265,0,0.0,40598.450,0.0
2021-01-31,185987,0.00,319583.0,0.0,0,0,31282.416,0.0000,0,0.0,39146.442,0.0
2021-02-28,194389,0.00,321869.0,0.0,15056,15000,12776.600,-23292.6411,0,0.0,47455.140,0.0


# Calculate Returns

#### Define functions to calculate returns

In [101]:
def get_returns(fund_num, input_df):
    fund = my_funds[fund_num]                               # pick fund
    trans = my_funds_trans[fund_num]                        # pick corresponding transactions

    df_fund = input_df[fund].reset_index()                     # extract relevant fund data
    df_trans = input_df[trans].reset_index()                   # extract relevant transaction data

    investment_date_index = df_fund.ne(0).idxmax()[fund]    # Index of row with first non-zero value in fund (initial investment)
    df_fund = df_fund.iloc[investment_date_index:]          # Trim leading rows with zeros from fund data
    df_trans = df_trans.iloc[investment_date_index:]        # Trim leading rows with zeros from transaction data
    df_all = df_fund.join(df_trans[trans])                  # Combine fund and transaction data
    df_all.set_index('index', inplace=True)                 # Reset index to dates
    df_all.index.name = None                                # Remove 'index' heading from index

    investment_periods = len(input_df) - investment_date_index # Number of periods fund has been held

    fund_returns = defaultdict(pd.DataFrame)
    fund_return = df_all                                    #pull a fund out of data frame
    
    # Return calcluations
    fund_return['months'] = range(1, 1+len(fund_return))     # number of holding period
    
    fund_return["1m_return"] = round(100*(fund_return[fund]-fund_return[fund].shift(1)-fund_return[trans])/(fund_return[fund].shift(1)),2)
    
    # Set initial investment for benchmarking. Currently using $100
    benchmark = 100
    if(fund_return[trans][0]>0):
        dollar_growth = [benchmark*(1+((fund_return[fund][0]-fund_return[trans][0])/fund_return[trans][0]))]
    else:
        dollar_growth = [benchmark]     # Fix for portfolios that have no holdings
    
    for i in range(1,len(fund_return['1m_return'])):
        dollar_growth.append(dollar_growth[i-1]*(1+fund_return['1m_return'][i]/100))
    
    fund_return['$1_growth'] = dollar_growth
    # fund_return['$1_growth'] = fund_return['$1_growth'].shift(1)*(1+fund_return["1m_return"]/100)
    fund_return["3m_return"] = round(100*((fund_return['$1_growth']-fund_return['$1_growth'].shift(3))/fund_return['$1_growth'].shift(3)),2)
    fund_return["6m_return"] = round(100*((fund_return['$1_growth']-fund_return['$1_growth'].shift(6))/fund_return['$1_growth'].shift(6)),2)
    fund_return["1y_return"] = round(100*((fund_return['$1_growth']-fund_return['$1_growth'].shift(12))/fund_return['$1_growth'].shift(12)),2)
    fund_return["2y_return"] = round(100*((fund_return['$1_growth']-fund_return['$1_growth'].shift(24))/fund_return['$1_growth'].shift(24)),2)
    fund_return["2y_return_annualized"] = round(100*((1+fund_return["2y_return"]/100)**(1/2)-1),2)
    fund_return["3y_return"] = round(100*((fund_return['$1_growth']-fund_return['$1_growth'].shift(36))/fund_return['$1_growth'].shift(36)),2)
    fund_return["3y_return_annualized"] = round(100*((1+fund_return["3y_return"]/100)**(1/3)-1),2)
    fund_return["5y_return"] = round(100*((fund_return['$1_growth']-fund_return['$1_growth'].shift(60))/fund_return['$1_growth'].shift(60)),2)
    fund_return["5y_return_annualized"] = round(100*((1+fund_return["5y_return"]/100)**(1/5)-1),2)
    fund_return["SI_return"] = round(100*((fund_return['$1_growth']-fund_return['$1_growth'][0])/fund_return['$1_growth'][0]),2)
    fund_return['annualized_return'] = round(100*(fund_return['$1_growth']**(12/fund_return['months'])-1),2)


    #########
    # # Taken out Mar 16    
    # trans_neg = fund_return[trans].where(fund_return[trans]<0,0)
    # trans_pos = fund_return[trans].where(fund_return[trans]>0,0)
    # fund_return['money_in_sum'] = trans_pos.expanding(min_periods=1).sum()
    # fund_return['money_out_sum'] = -1*trans_neg.expanding(min_periods=1).sum()
    #     
    # fund_return['net_gains'] = -fund_return['money_in_sum'] + fund_return[fund] + fund_return['money_out_sum']
    # fund_return['unrealized_gains'] = np.where(fund_return[fund] <= 0, 0, (fund_return['net_gains'] - fund_return['money_out_sum']))
    ########

    fund_return['volatility'] = 100*((fund_return['1m_return']/100).expanding(min_periods=1).std())
    fund_return['annualized_volatility'] = fund_return['volatility']*np.sqrt(fund_return['months'])

    fund_returns[fund] = fund_return

    return fund_returns[fund]


def compile_returns(input_df):
    # Extract fund names and transaction names from df
    
    global fundcols
    fundcols = list(input_df.columns.values) 
    global my_funds
    my_funds = fundcols[::2]
    global my_funds_trans
    my_funds_trans = fundcols[1::2]

    # Use a for loop to run get_return on each fund in my_funds & store each df in a list
    global returns_list     #<-- making this global allows you to access the intermediate result later
    returns_list = []

    for i in range(0,len(my_funds)):
        returns = get_returns(i, input_df)
        returns_list.append(returns)
    

    # Use a for loop to select the last row in every df in returns_list
    tails_list = []

    for i in range(0,len(returns_list)):
        tails = returns_list[i].tail(1)
        tails.rename(columns={ tails.columns[0]: "Position" }, inplace = True)
        dropcol = my_funds_trans[i]
        tails.drop(columns=dropcol, inplace=True)
        tails_list.append(tails)
    
    for i in tails_list:
        i.columns = ['Position_Value','Months','1M_Return','$100_Growth','3M_Return','6M_Return','1Y_Return','2Y_Return','2Y_Return_Annualized','3Y_Return','3Y_Return_Annualized','5Y_Return','5Y_Return_Annualized','SI_Return','Annualized_Return','Volatility', 'Annualized_Volatility']

    summary_df = pd.concat(tails_list, axis=0)

    summary_df.insert(0, column = 'Position_Name', value = my_funds)

    # summary_df = summary_df[summary_df['Position_Value'] != 0]  # Drop outdated positions

    summary_df.fillna('-', inplace=True)    # Replace NaNs with "-"

    
    return summary_df

def compile_returns_individual(input_df):
    # Extract fund names and transaction names from df
    
    global fundcols
    fundcols = list(input_df.columns.values) 
    global my_funds
    my_funds = fundcols[2::7]      #<--- we need to get my_funds variable to include the 'Total' column and then it will be identical to what existed before we added the new columns. I think the issue has                                                    something to do with the fact that the funcols variable is iterative. 
    #print(type(my_funds))
    #my_fung = fundcols[2::7]
    #my_funds = my_fung.append(fundcols[0])
    global my_funds_trans
    my_funds_trans = fundcols[1::7]

    # Use a for loop to run get_return on each fund in my_funds & store each df in a list
    global returns_list     #<-- making this global allows you to access the intermediate result later
    returns_list = []

    for i in range(0,len(my_funds)):
        returns = get_returns(i, input_df)
        returns_list.append(returns)
    

    # Use a for loop to select the last row in every df in returns_list
    tails_list = []

    for i in range(0,len(returns_list)):
        tails = returns_list[i].tail(1)
        tails.rename(columns={ tails.columns[0]: "Position" }, inplace = True)
        dropcol = my_funds_trans[i]
        tails.drop(columns=dropcol, inplace=True)
        tails_list.append(tails)
    
    for i in tails_list:
        i.columns = ['Position_Value','Months','1M_Return','$100_Growth','3M_Return','6M_Return','1Y_Return','2Y_Return','2Y_Return_Annualized','3Y_Return','3Y_Return_Annualized','5Y_Return','5Y_Return_Annualized','SI_Return','Annualized_Return','Volatility', 'Annualized_Volatility']

    summary_df = pd.concat(tails_list, axis=0)

    summary_df.insert(0, column = 'Position_Name', value = my_funds)

    # summary_df = summary_df[summary_df['Position_Value'] != 0]  # Drop outdated positions

    summary_df.fillna('-', inplace=True)    # Replace NaNs with "-"

    
    return summary_df



In [102]:
test_fundcol = list(holdings_summary.columns.values)
test_fundcol


['Growth (Total)',
 'Growth (Total)[trans]',
 'Market Neutral (Total)',
 'Market Neutral (Total)[trans]',
 'Sustainability (Total)',
 'Sustainability (Total)[trans]',
 'Thematic (Total)',
 'Thematic (Total)[trans]',
 'Special Situations (Total)',
 'Special Situations (Total)[trans]',
 'Conservative Tactical (Total)',
 'Conservative Tactical (Total)[trans]']

## Holdings Summary Returns

In [103]:
returns_holdings_summary = compile_returns(holdings_summary)
returns_holdings_summary

,Position_Name,Position_Value,Months,1M_Return,$100_Growth,3M_Return,6M_Return,1Y_Return,2Y_Return,2Y_Return_Annualized,3Y_Return,3Y_Return_Annualized,5Y_Return,5Y_Return_Annualized,SI_Return,Annualized_Return,Volatility,Annualized_Volatility
2021-03-31,Growth (Total),139663.00,84,4.17,179.646,15.18,19.83,53.99,55.89,24.86,53.91,15.46,86.63,13.29,79.65,109.92,2.65477,24.3314
2021-03-31,Market Neutral (Total),326844.00,109,1.55,117.138,4.6,9.22,13.03,18.92,9.05,17.39,5.49,20.79,3.85,16.98,68.95,1.41806,14.805
2021-03-31,Sustainability (Total),14767.00,2,-1.92,98.4462,-,-,-,-,-,-,-,-,-,-1.92,9.10317e+13,-,-
2021-03-31,Thematic (Total),11583.40,9,-9.34,372.695,4.99,40.38,-,-,-,-,-,-,-,215.73,268108,22.9089,68.7268
2021-03-31,Special Situations (Total),0.00,70,-,-,-,-,-,-,-,-,-,-,-,-,-,4.27517,35.7686
2021-03-31,Conservative Tactical (Total),111120.38,110,134.16,444.448,173.69,90.84,186.82,43.39,19.75,14.26,4.54,-2.9,-0.59,344.45,94.47,58.8963,617.709


#### Export summary dataframe

In [104]:
returns_holdings_summary.to_csv('returns_summary.csv')

In [44]:
# Print total portfolio histories
for fund in range(0,len(my_funds)-2):   # Only print Growth, Market Neutral, Sustainability, Thematic
    filename = 'returns_'+my_funds[fund]+'.csv'
    # filename = 'returns_total/returns_'+my_funds[fund]+'.csv'
    returns_list[fund].to_csv(filename)

## Individual Fund Returns

In [154]:
wanted_sheets = sheet_names[1:11]
wanted_sheets

['growth_cad',
 'growth_usd',
 'market_neutral_cad',
 'market_neutral_usd',
 'sustainability_cad',
 'sustainability_usd',
 'thematic_cad',
 'thematic_usd',
 'special_situations_cad',
 'special_situations_usd']

In [155]:
# Load data

individual_holdings = defaultdict(pd.DataFrame)
filename = 'holdings-newcol_DD.xlsx'
for sheet_name in wanted_sheets:
    individual_holdings[sheet_name] = pd.read_excel(filename, sheet_name = sheet_name, header = 5)
    individual_holdings[sheet_name].drop(columns=['Month','Year'], inplace=True)

    # Reset index to date
    individual_holdings[sheet_name].set_index('Date', inplace=True)
    individual_holdings[sheet_name].index.name = None

    # Replace NaNs with 0s (No position = $0 position)
    individual_holdings[sheet_name] = individual_holdings[sheet_name].fillna(0)

In [156]:
individual_holdings['thematic_usd']

,Total,Transactions,Zillow Group Inc.,Zillow Group Inc. [trans],Zillow Group Inc. [Shares],Zillow Group Inc. [Total Quantity Balance],Zillow Group Inc. [Cost Basis],Zillow Group Inc. [Unrealized Gain (Loss)],Zillow Group Inc. [Realized Gain (Loss)],SQUARE Inc.,SQUARE Inc.[trans],SQUARE Inc.[Shares],SQUARE Inc.[Total Quantity Balance],SQUARE Inc.[Cost Basis],SQUARE Inc.[Unrealized Gain (Loss)],SQUARE Inc.[Realized Gain (Loss)],Sea Limited,Sea Limited [trans],Sea Limited [Shares],Sea Limited [Total Quantity Balance],Sea Limited [Cost Basis],Sea Limited [Unrealized Gain (Loss)],Sea Limited [Realized Gain (Loss)],Mitek Systems Inc,Mitek Systems Inc[trans],Mitek Systems Inc[Shares],Mitek Systems Inc[Total Quantity Balance],Mitek Systems Inc[Cost Basis],Mitek Systems Inc[Unrealized Gain (Loss)],Mitek Systems Inc[Realized Gain (Loss)],Mercadolibre Inc,Mercadolibre Inc[trans],Mercadolibre Inc[Shares],Mercadolibre Inc[Total Quantity Balance],Mercadolibre Inc[Cost Basis],Mercadolibre Inc[Unrealized Gain (Loss)],Mercadolibre Inc[Realized Gain (Loss)],ABBVIE INC,ABBVIE INC[trans],ABBVIE INC[Shares],ABBVIE INC[Total Quantity Balance],ABBVIE INC[Cost Basis],ABBVIE INC[Unrealized Gain (Loss)],ABBVIE INC[Realized Gain (Loss)],ALECTOR INC,ALECTOR INC[trans],ALECTOR INC[Shares],ALECTOR INC[Total Quantity Balance],ALECTOR INC[Cost Basis],ALECTOR INC[Unrealized Gain (Loss)],ALECTOR INC[Realized Gain (Loss)],ZYMEWORKS INC,ZYMEWORKS INC[trans],ZYMEWORKS INC[Shares],ZYMEWORKS INC[Total Quantity Balance],ZYMEWORKS INC[Cost Basis],ZYMEWORKS INC[Unrealized Gain (Loss)],ZYMEWORKS INC[Realized Gain (Loss)]
2012-01-31,0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-02-29,0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-03-31,0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-04-30,0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-05-31,0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-30,16052,0.00,2911.0,0.00,0.0,0.0,0.0,0.0,0.0,1899.0,0.00,0.0,0.0,0.0,0.0,0.0,2164.0,0.0,0.0,0.0,0.0,0.0,0.0,4418.0,0.00,0.0,0.0,0.0,0.0,0.0,4660.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-31,20849,1042.06,3505.0,0.00,0.0,0.0,0.0,0.0,0.0,1959.0,0.00,0.0,0.0,0.0,0.0,0.0,2389.0,0.0,0.0,0.0,0.0,0.0,0.0,3272.0,-3728.23,0.0,0.0,0.0,0.0,0.0,5026.0,0.00,0.0,0.0,0.0,0.0,0.0,2357.0,2263.31,0.0,0.0,0.0,0.0,0.0,923.0,962.58,0.0,0.0,0.0,0.0,0.0,1418.0,1544.4,0.0,0.0,0.0,0.0,0.0
2021-01-31,20704,0.00,3522.0,0.00,0.0,0.0,0.0,0.0,0.0,1944.0,0.00,0.0,0.0,0.0,0.0,0.0,2601.0,0.0,0.0,0.0,0.0,0.0,0.0,2972.0,0.00,0.0,0.0,0.0,0.0,0.0,5339.0,0.00,0.0,0.0,0.0,0.0,0.0,2284.0,0.00,0.0,0.0,0.0,0.0,0.0,1027.0,0.00,0.0,0.0,0.0,0.0,0.0,1015.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-02-28,7380,-17284.93,0.0,-5569.33,0.0,0.0,0.0,0.0,0.0,0.0,-2487.31,0.0,0.0,0.0,0

In [157]:
#individual_holdings['thematic_usd']
#b = [0,2,9,16,23,30,37,44,51,58,65,72,79,86,93,100,107,114]
#a = np.array([1,5,6])
test2_fundcol = list(individual_holdings['thematic_usd'].columns.values)

ttt = test2_fundcol[2::7]
ttt.insert(0,'Total')
ttt

['Total',
 'Zillow Group Inc. ',
 'SQUARE Inc.',
 'Sea Limited ',
 'Mitek Systems Inc',
 'Mercadolibre Inc',
 'ABBVIE INC',
 'ALECTOR INC',
 'ZYMEWORKS INC']

In [83]:
test3_fundcol = list(individual_holdings['thematic_usd'].columns.values)
test3_fundcol[0]

'Total'

In [158]:
# Get Returns

returns_individual_holdings = defaultdict(pd.DataFrame)

for sheet_name in wanted_sheets:
    #returns_individual_holdings[sheet_name] = compile_returns(individual_holdings[sheet_name])
    returns_individual_holdings[sheet_name] = compile_returns_individual(individual_holdings[sheet_name])


In [125]:
def compile_returns_total(input_df):
    # Extract fund names and transaction names from df
    
    global fundcols
    fundcols = list(input_df.columns.values) 
    global my_funds
    my_funds = fundcols[:2:7]      #gets us a Totals column now
    #my_fung = fundcols[2::7]
    #my_funds = my_fung.append(fundcols[0])
    global my_funds_trans
    my_funds_trans = fundcols[1::7]

    # Use a for loop to run get_return on each fund in my_funds & store each df in a list
    global returns_list     #<-- making this global allows you to access the intermediate result later
    returns_list = []

    for i in range(0,len(my_funds)):
        returns = get_returns(i, input_df)
        returns_list.append(returns)
    

    # Use a for loop to select the last row in every df in returns_list
    tails_list = []

    for i in range(0,len(returns_list)):
        tails = returns_list[i].tail(1)
        tails.rename(columns={ tails.columns[0]: "Position" }, inplace = True)
        dropcol = my_funds_trans[i]
        tails.drop(columns=dropcol, inplace=True)
        tails_list.append(tails)
    
    for i in tails_list:
        i.columns = ['Position_Value','Months','1M_Return','$100_Growth','3M_Return','6M_Return','1Y_Return','2Y_Return','2Y_Return_Annualized','3Y_Return','3Y_Return_Annualized','5Y_Return','5Y_Return_Annualized','SI_Return','Annualized_Return','Volatility', 'Annualized_Volatility']

    summary_df = pd.concat(tails_list, axis=0)

    summary_df.insert(0, column = 'Position_Name', value = my_funds)

    # summary_df = summary_df[summary_df['Position_Value'] != 0]  # Drop outdated positions

    summary_df.fillna('-', inplace=True)    # Replace NaNs with "-"

    
    return summary_df

returns_total_holdings = defaultdict(pd.DataFrame)

for sheet_name in wanted_sheets:
    #returns_individual_holdings[sheet_name] = compile_returns(individual_holdings[sheet_name])
    returns_total_holdings[sheet_name] = compile_returns_total(individual_holdings[sheet_name])





In [159]:
# Look at sample
Total_row = returns_total_holdings['thematic_usd']
return_file = returns_individual_holdings['thematic_usd']
pd.concat([Total_row, return_file], ignore_index=True)

,Position_Name,Position_Value,Months,1M_Return,$100_Growth,3M_Return,6M_Return,1Y_Return,2Y_Return,2Y_Return_Annualized,3Y_Return,3Y_Return_Annualized,5Y_Return,5Y_Return_Annualized,SI_Return,Annualized_Return,Volatility,Annualized_Volatility
0,Total,7240.0,6,-1.9,156.611,16.05,-,-,-,-,-,-,-,-,64.3,2.45259e+06,11.585265,28.377988
1,Zillow Group Inc.,0.0,6,-,-,-,-,-,-,-,-,-,-,-,-,-,194.851627,477.287062
2,SQUARE Inc.,0.0,6,-,-,-,-,-,-,-,-,-,-,-,-,-,58.825491,144.092438
3,Sea Limited,0.0,6,-,-,-,-,-,-,-,-,-,-,-,-,-,55.656808,136.330779
4,Mitek Systems Inc,2683.0,6,-4.59,57.461,-18,-,-,-,-,-,-,-,-,-42.54,330076,9.037243,22.136635
5,Mercadolibre Inc,0.0,6,-,-,-,-,-,-,-,-,-,-,-,-,-,57.854725,141.714556
6,ABBVIE INC,2381.0,4,0.46,101.016,1.02,-,-,-,-,-,-,-,-,1.02,1.03078e+08,3.435758,6.871516
7,ALECTOR INC,1229.0,4,10.82,133.15,33.15,-,-,-,-,-,-,-,-,33.15,2.36058e+08,1.783825,3.567651
8,ZYMEWORKS INC,947.0,4,-13.04,66.7837,-33.22,-,-,-,-,-,-,-,-,-33.22,2.97858e+07,17.912088,35.824176


In [160]:
# Write all individual returns to csv files (SHOULD THIS BE GOING TO SINGLE EXCEL?)
for sheet_name in wanted_sheets:
    #filename = 'returns/returns_individual-'+sheet_name+'.csv'
    filename = 'returns_individual-'+sheet_name+'.csv'
    returns_individual_holdings[sheet_name].insert(0, column='Portfolio', value=sheet_name)
    #returns_individual_holdings[sheet_name].insert(0, column='Portfolio', value=sheet_name, allow_duplicates=True)
    returns_individual_holdings[sheet_name].to_csv(filename) 
    

# Inspect individual portfolio and individual holding

In [161]:
examine_holding = compile_returns_individual(individual_holdings['growth_cad'])
row_total = returns_total_holdings['growth_cad']
examine_holding =pd.concat([row_total, examine_holding], ignore_index=True)
examine_holding

,Position_Name,Position_Value,Months,1M_Return,$100_Growth,3M_Return,6M_Return,1Y_Return,2Y_Return,2Y_Return_Annualized,3Y_Return,3Y_Return_Annualized,5Y_Return,5Y_Return_Annualized,SI_Return,Annualized_Return,Volatility,Annualized_Volatility
0,Total,139663.0,84,4.17,179.646,15.18,19.83,53.99,55.89,24.86,53.91,15.46,86.63,13.29,79.65,109.92,2.654774,24.331407
1,"DYNAMIC GLOBAL GROWTH OPPORTUNITIES FUND,SER F...",42025.0,23,141.47,-470.69,163.72,194.34,320.8,-,-,-,-,-,-,-570.69,-,55.741361,267.326174
2,LYNWOOD OPPORTUNITIES FUND CL F (1033),63105.0,18,14,252.393,38.14,43.34,141.76,-,-,-,-,-,-,152.39,3893.79,8.351683,35.433190
3,NEWGEN EQUITY LONG-SHORT FUND LP CLASS F (104),34533.0,18,2.66,138.137,14.59,18.93,34.83,-,-,-,-,-,-,38.14,2572.22,2.933156,12.444326
4,EHP ADVANTAGE FUND CL F 2013 LEAD SERIES (101F),0.0,72,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,12.096030,102.638218
5,VANTAGE CLASS F SERIES I 2014 SERIES 1 (201),0.0,56,-,-,-,-,-,-,-,-,-,-,-,-,-,17.801414,133.213582
6,FG GLOBAL OPPORTUNITIES CLASS F SUBS (900),0.0,84,-,-,-,-,-,-,-,-,-,-,-,-,-,22.917153,210.039176
7,VERTEX FUND CLASS F (301),0.0,83,-,-,-,-,-,-,-,-,-,-,-,-,-,30.415916,277.102180


In [ ]:
examine_holding_individual = returns_list[1]
examine_holding_individual